# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    text = set(text)
    vocab_to_int = {word : i for i, word in enumerate(text)}
    
    int_to_vocab = {i : word for i, word in enumerate(text)}

    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuations = {'.': '||dot||',',':'||comma||',
    '"' : '||quote||',
    ';' : '||semicolon||',
    '!' : '||exclamation_mark||',
    '?' : '||question_mark||',
    '(' : '||left_parenthesis||',
    ')' : '||right_parenthesis||',
    '--' : '||Dash||',
    '\n' : '||Return||'}

    return punctuations

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [9]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    theInput = tf.placeholder(tf.int32,[None,None], name='input')
    targets = tf.placeholder(tf.int32,[None,None], name='target')
    lr = tf.placeholder(tf.float32,name = 'LearningRate')
    return theInput, targets, lr


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [10]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    
   
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    
    # Getting an initial state of all zeros
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name = 'initial_state')
    # TODO: Implement Function
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embed = tf.Variable(tf.random_uniform((vocab_size, embed_dim) , -1, 1))
    embedding  = tf.nn.embedding_lookup(embed, input_data)
    return embedding


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [12]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    output, final_state = tf.nn.dynamic_rnn(cell, inputs,dtype=tf.float32)
    final_state = tf.identity(final_state,name='final_state')
    return output, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    inputs , targets, lr = get_inputs()
    embedding = get_embed(vocab_size=vocab_size, input_data=input_data,embed_dim=embed_dim)
    outputs, final_state = build_rnn(cell=cell, inputs= embedding)
    
    logits = tf.contrib.layers.fully_connected(outputs, activation_fn=None, num_outputs = vocab_size)


    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [14]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    print( 'batch size = ', batch_size, ' and seq length = ', seq_length)
    
    print ('text leng = ', len(int_text))
    # TODO: Implement Function
    length = len(int_text)
    real_batch_size = (batch_size * seq_length)
    total_batches = length // real_batch_size
    final_text_length = total_batches * real_batch_size
    
    print('final_text_length = ', final_text_length)
    print(total_batches, ' IS THE TOTAL BATCHES')
    
    int_text = int_text[:final_text_length]
    length = len(int_text)
    
    inputArray = []
    targetArray = []

    
 
    for i in range(0, length - seq_length + 1, seq_length):
        temp = []
        for ii in range(seq_length):
            temp.append(int_text[i+ii])
        inputArray.append(temp)


    for i in range(1, length - seq_length + 1, seq_length):
        temp = []
        for ii in range(seq_length):
            temp.append(int_text[i+ii])
        targetArray.append(temp)

    
    total_seq = (length-1)//seq_length


    indexOfLastItem = total_seq * seq_length


    toAppend = int_text[indexOfLastItem +1 : ]

    remnant = seq_length - len(toAppend)

    toAppend.extend(int_text[:remnant])
    targetArray.append(toAppend)

    
    print (inputArray[-3:])
    print (targetArray[-3:])

    inputArray = np.array(inputArray)
    targetArray = np.array(targetArray)
    print (inputArray.shape)
    print ( targetArray.shape)

    myBatches = []

    for c in range(total_batches):
        bat_i = []
        bat_t = []
        for i in range(batch_size):
            bat_i.append(inputArray[c + i * total_batches])
            bat_t.append(targetArray[c + i * total_batches])

        temp_bat = []
        temp_bat.append(bat_i)
        temp_bat.append(bat_t)
        myBatches.append(temp_bat)

    #print(myBatches)

    myBatches = np.array(myBatches)
    print (myBatches.shape)
   
    
    return myBatches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

batch size =  128  and seq length =  5
text leng =  5000
final_text_length =  4480
7  IS THE TOTAL BATCHES
[[4465, 4466, 4467, 4468, 4469], [4470, 4471, 4472, 4473, 4474], [4475, 4476, 4477, 4478, 4479]]
[[4466, 4467, 4468, 4469, 4470], [4471, 4472, 4473, 4474, 4475], [4476, 4477, 4478, 4479, 0]]
(896, 5)
(896, 5)
(7, 2, 128, 5)
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [17]:
# Number of Epochs
num_epochs = 200
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 128
# Embedding Dimension Size
embed_dim = 250
# Sequence Length
seq_length = 6
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 20

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [19]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

batch size =  128  and seq length =  6
text leng =  69100
final_text_length =  68352
89  IS THE TOTAL BATCHES
[[5, 4697, 4818, 5532, 1307, 4127], [3422, 3239, 2296, 3240, 4697, 2310], [764, 1702, 3536, 6248, 4762, 5532]]
[[4697, 4818, 5532, 1307, 4127, 3422], [3239, 2296, 3240, 4697, 2310, 764], [1702, 3536, 6248, 4762, 5532, 4009]]
(11392, 6)
(11392, 6)
(89, 2, 128, 6)
Epoch   0 Batch    0/89   train_loss = 8.820
Epoch   0 Batch   20/89   train_loss = 6.083
Epoch   0 Batch   40/89   train_loss = 5.505
Epoch   0 Batch   60/89   train_loss = 5.350
Epoch   0 Batch   80/89   train_loss = 5.030
Epoch   1 Batch   11/89   train_loss = 4.838
Epoch   1 Batch   31/89   train_loss = 4.624
Epoch   1 Batch   51/89   train_loss = 4.569
Epoch   1 Batch   71/89   train_loss = 4.729
Epoch   2 Batch    2/89   train_loss = 4.377
Epoch   2 Batch   22/89   train_loss = 4.516
Epoch   2 Batch   42/89   train_loss = 4.313
Epoch   2 Batch   62/89   train_loss = 4.230
Epoch   2 Batch   82/89   train_loss = 4.0

Epoch  39 Batch    9/89   train_loss = 1.273
Epoch  39 Batch   29/89   train_loss = 1.324
Epoch  39 Batch   49/89   train_loss = 1.197
Epoch  39 Batch   69/89   train_loss = 1.173
Epoch  40 Batch    0/89   train_loss = 1.296
Epoch  40 Batch   20/89   train_loss = 1.266
Epoch  40 Batch   40/89   train_loss = 1.299
Epoch  40 Batch   60/89   train_loss = 1.225
Epoch  40 Batch   80/89   train_loss = 1.388
Epoch  41 Batch   11/89   train_loss = 1.307
Epoch  41 Batch   31/89   train_loss = 1.320
Epoch  41 Batch   51/89   train_loss = 1.312
Epoch  41 Batch   71/89   train_loss = 1.179
Epoch  42 Batch    2/89   train_loss = 1.241
Epoch  42 Batch   22/89   train_loss = 1.219
Epoch  42 Batch   42/89   train_loss = 1.280
Epoch  42 Batch   62/89   train_loss = 1.366
Epoch  42 Batch   82/89   train_loss = 1.202
Epoch  43 Batch   13/89   train_loss = 1.170
Epoch  43 Batch   33/89   train_loss = 1.237
Epoch  43 Batch   53/89   train_loss = 1.270
Epoch  43 Batch   73/89   train_loss = 1.147
Epoch  44 

Epoch  80 Batch   20/89   train_loss = 0.997
Epoch  80 Batch   40/89   train_loss = 1.074
Epoch  80 Batch   60/89   train_loss = 1.034
Epoch  80 Batch   80/89   train_loss = 1.094
Epoch  81 Batch   11/89   train_loss = 1.059
Epoch  81 Batch   31/89   train_loss = 1.125
Epoch  81 Batch   51/89   train_loss = 1.111
Epoch  81 Batch   71/89   train_loss = 1.026
Epoch  82 Batch    2/89   train_loss = 1.009
Epoch  82 Batch   22/89   train_loss = 1.038
Epoch  82 Batch   42/89   train_loss = 1.106
Epoch  82 Batch   62/89   train_loss = 1.167
Epoch  82 Batch   82/89   train_loss = 1.044
Epoch  83 Batch   13/89   train_loss = 0.975
Epoch  83 Batch   33/89   train_loss = 1.074
Epoch  83 Batch   53/89   train_loss = 1.164
Epoch  83 Batch   73/89   train_loss = 1.065
Epoch  84 Batch    4/89   train_loss = 1.071
Epoch  84 Batch   24/89   train_loss = 1.103
Epoch  84 Batch   44/89   train_loss = 1.070
Epoch  84 Batch   64/89   train_loss = 1.094
Epoch  84 Batch   84/89   train_loss = 1.127
Epoch  85 

Epoch 121 Batch   31/89   train_loss = 0.937
Epoch 121 Batch   51/89   train_loss = 0.939
Epoch 121 Batch   71/89   train_loss = 0.860
Epoch 122 Batch    2/89   train_loss = 0.871
Epoch 122 Batch   22/89   train_loss = 0.893
Epoch 122 Batch   42/89   train_loss = 0.903
Epoch 122 Batch   62/89   train_loss = 0.956
Epoch 122 Batch   82/89   train_loss = 0.888
Epoch 123 Batch   13/89   train_loss = 0.813
Epoch 123 Batch   33/89   train_loss = 0.858
Epoch 123 Batch   53/89   train_loss = 0.933
Epoch 123 Batch   73/89   train_loss = 0.856
Epoch 124 Batch    4/89   train_loss = 0.870
Epoch 124 Batch   24/89   train_loss = 0.912
Epoch 124 Batch   44/89   train_loss = 0.874
Epoch 124 Batch   64/89   train_loss = 0.873
Epoch 124 Batch   84/89   train_loss = 0.921
Epoch 125 Batch   15/89   train_loss = 0.915
Epoch 125 Batch   35/89   train_loss = 0.880
Epoch 125 Batch   55/89   train_loss = 0.872
Epoch 125 Batch   75/89   train_loss = 0.934
Epoch 126 Batch    6/89   train_loss = 0.956
Epoch 126 

Epoch 162 Batch   42/89   train_loss = 1.431
Epoch 162 Batch   62/89   train_loss = 1.409
Epoch 162 Batch   82/89   train_loss = 1.318
Epoch 163 Batch   13/89   train_loss = 1.324
Epoch 163 Batch   33/89   train_loss = 1.256
Epoch 163 Batch   53/89   train_loss = 1.261
Epoch 163 Batch   73/89   train_loss = 1.247
Epoch 164 Batch    4/89   train_loss = 1.247
Epoch 164 Batch   24/89   train_loss = 1.259
Epoch 164 Batch   44/89   train_loss = 1.265
Epoch 164 Batch   64/89   train_loss = 1.235
Epoch 164 Batch   84/89   train_loss = 1.291
Epoch 165 Batch   15/89   train_loss = 1.246
Epoch 165 Batch   35/89   train_loss = 1.188
Epoch 165 Batch   55/89   train_loss = 1.099
Epoch 165 Batch   75/89   train_loss = 1.199
Epoch 166 Batch    6/89   train_loss = 1.241
Epoch 166 Batch   26/89   train_loss = 1.136
Epoch 166 Batch   46/89   train_loss = 1.099
Epoch 166 Batch   66/89   train_loss = 1.160
Epoch 166 Batch   86/89   train_loss = 1.063
Epoch 167 Batch   17/89   train_loss = 1.118
Epoch 167 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [21]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [22]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [23]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    
# The following solution also work.. but it is ugly So I preferred to convert the array 
# to a list 

#     index = np.where(probabilities == max(probabilities))
#     print (index)
#     print(int_to_vocab[index[0][0]])
    
#     return int_to_vocab[index[0][0]]


    index = list(probabilities).index(max(probabilities))
    print (index)
    print(int_to_vocab[index])
    
    return int_to_vocab[index]




"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

1
is
Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [24]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
5718
||left_parenthesis||
536
lying
3089
||right_parenthesis||
606
oh
2014
||comma||
857
it
4823
is
3175
a
3263
home
5532
||dot||
4987
||return||
5207
barney_gumble:
5718
||left_parenthesis||
1285
tipsy
3089
||right_parenthesis||
5012
homer
2014
||comma||
4147
this
3544
little
4823
is
148
my
4685
exact
6310
double
5532
||dot||
5532
||dot||
5532
||dot||
5718
||left_parenthesis||
5196
steely-eyed
4378
after
46
he
863
beat
3089
||right_parenthesis||
3422
that
1307
i
5826
gave
4697
me
1464
hold
6600
on
764
the
5
love
5501
of
5532
||dot||
4987
||return||
2465
homer_simpson:
3515
what
2342
are
3239
you
2014
||comma||
1871
moe
5952
||question_mark||
4987
||return||
2465
homer_simpson:
5718
||left_parenthesis||
2305
loud
3457
whisper
3089
||right_parenthesis||
1307
i
5778
don't
218
know
3515
what
5952
||question_mark||
5718
||left_parenthesis||
2616
mad
3089
||right_parenthesis||
6746
hey
2014
||comma||
1871
moe
2014
||comma||
3239
you
3995
wann

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.